In [12]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Embedding, Linear

import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import SAGEConv
from torch_geometric.nn.conv.gcn_conv import gcn_norm


In [13]:
from utils import *

parameters = {
    "hidden_dims": [16,8,8],
    "df_name": 'mindsteps_set_full',
    "epochs": 10000,
    "learning_rate": 0.005,
    'weight_decay': 0,
    'dropout': 0.4,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda',
    'batch_size': 1024
}
    

df = load_data_heterogeneous(parameters["df_name"])
data = create_data_object_heterogeneous(df)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [15]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('student', 'takes', 'code')],
    rev_edge_types=[('code', 'rev_takes', 'student')],
)(data)

# Generate the co-occurence matrix of movies<>movies:
metapath = [('code', 'rev_takes', 'student'), ('student', 'takes', 'code')]
train_data = T.AddMetaPaths(metapaths=[metapath])(train_data)

# Apply normalization to filter the metapath:
_, edge_weight = gcn_norm(
    train_data['code', 'code'].edge_index,
    num_nodes=train_data['code'].num_nodes,
    add_self_loops=False,
)
edge_index = train_data['code', 'code'].edge_index[:, edge_weight > 0.002]

train_data['code', 'metapath_0', 'code'].edge_index = edge_index
val_data['code', 'metapath_0', 'code'].edge_index = edge_index
test_data['code', 'metapath_0', 'code'].edge_index = edge_index


class MovieGNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = SAGEConv(-1, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        return self.lin(x)


class UserGNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), hidden_channels)
        self.conv3 = SAGEConv((-1, -1), hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        movie_x = self.conv1(
            x_dict['code'],
            edge_index_dict[('code', 'metapath_0', 'code')],
        ).relu()

        user_x = self.conv2(
            (x_dict['code'], x_dict['student']),
            edge_index_dict[('code', 'rev_takes', 'student')],
        ).relu()

        user_x = self.conv3(
            (movie_x, user_x),
            edge_index_dict[('code', 'rev_takes', 'student')],
        ).relu()

        return self.lin(user_x)


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_src, z_dst, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_src[row], z_dst[col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, num_users, hidden_channels, out_channels):
        super().__init__()
        self.user_emb = Embedding(num_users, hidden_channels)
        self.user_encoder = UserGNNEncoder(hidden_channels, out_channels)
        self.movie_encoder = MovieGNNEncoder(hidden_channels, out_channels)
        self.decoder = EdgeDecoder(out_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = {}
        x_dict['student'] = self.user_emb(x_dict['student'])
        z_dict['student'] = self.user_encoder(x_dict, edge_index_dict)
        z_dict['code'] = self.movie_encoder(
            x_dict['code'],
            edge_index_dict[('code', 'metapath_0', 'code')],
        )
        return self.decoder(z_dict['student'], z_dict['code'], edge_label_index)


model = Model(data['student'].num_nodes, hidden_channels=64, out_channels=64)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(
        train_data.x_dict,
        train_data.edge_index_dict,
        train_data['student', 'code'].edge_label_index,
    )
    loss = F.mse_loss(out, train_data['student', 'code'].edge_label)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    out = model(
        data.x_dict,
        data.edge_index_dict,
        data['student', 'code'].edge_label_index,
    ).clamp(min=0, max=5)
    rmse = F.mse_loss(out, data['student', 'code'].edge_label).sqrt()
    return float(rmse)


for epoch in range(1, 701):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:04d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

RuntimeError: CUDA error: insufficient resources when calling `cusparseSpGEMM_workEstimation( handle, opA, opB, &alpha, matA, matB, &beta, matC, computeType, CUSPARSE_SPGEMM_DEFAULT, spgemmDesc, &bufferSize1, dBuffer1)`